# Delta Connectors
https://delta.io/integrations

# Data Consolidation
* Bringing in 
  * All data types: Structured, Semi-Structured, Un-Structured 
  * All frequencies: batch, micro-batch, continuous streaming
  * All sizes: Megabyt/Gigabyte/Terabyte/Petabyte scale

## Breaking Data Silos
* Whole is greater than sum of its parts
* Full cross-section of all data dimensions in a single place for analytics

In [0]:
# Clean prior run data files
dbutils.fs.rm('/tmp/ch-5/', True)

# Drop & recreate database
spark.sql("DROP DATABASE IF EXISTS ch_5 CASCADE")
spark.sql("CREATE DATABASE ch_5 ")
spark.sql("USE ch_5")

Out[47]: DataFrame[]

### Structured data

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([ \
    StructField("year", IntegerType(),True), \
    StructField("month",IntegerType(),True), \
    StructField("day",IntegerType(),True), \
    StructField("customer", StringType(), True), \
    StructField("sale_id", StringType(), True), \
    StructField("total_cost", FloatType(), True) \
  ])
strutured_data = [(2022, 3,2, "Jim Smith","M12SD", 30.37),
             (2022, 6,2, "Jill King","K127D", 50.5),
             (2022, 1,2, "Jack Morris","12PSD",100.0)
  ]
df = spark.createDataFrame(data=strutured_data,schema=schema)
df.printSchema()
df.show(truncate=False)
delta_flower_tx_path = '/tmp/ch-5/delta_transaction'
df.write.format("delta").mode('overwrite').save(delta_flower_tx_path)
spark.sql("CREATE TABLE Flower_Transaction USING DELTA LOCATION '" + delta_flower_tx_path + "'")

root
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- customer: string (nullable = true)
-- sale_id: string (nullable = true)
-- total_cost: float (nullable = true)

+----+-----+---+-----------+-------+----------+
year|month|day|customer |sale_id|total_cost|
+----+-----+---+-----------+-------+----------+
2022|3 |2 |Jim Smith |M12SD |30.37 |
2022|6 |2 |Jill King |K127D |50.5 |
2022|1 |2 |Jack Morris|12PSD |100.0 |
+----+-----+---+-----------+-------+----------+

Out[48]: DataFrame[]

### Other Formats
* Instead of CSV, the data can be other formats such as parquet, avro, ORC
* Let's simulate those data formats & read them in as delta

#### Avro

In [0]:
avro_path='/tmp/ch-5/avro'
df.write.format("avro").mode('overwrite').save(avro_path)

avro_to_delta_path = '/tmp/ch-5/avro_to_delta'
spark.read.format("avro").load(avro_path).write.format("delta").mode('overwrite').save(avro_to_delta_path)
avro_to_delta_df=spark.read.format("delta").load(avro_to_delta_path).show()

+----+-----+---+-----------+-------+----------+
year|month|day| customer|sale_id|total_cost|
+----+-----+---+-----------+-------+----------+
2022| 1| 2|Jack Morris| 12PSD| 100.0|
2022| 6| 2| Jill King| K127D| 50.5|
2022| 3| 2| Jim Smith| M12SD| 30.37|
+----+-----+---+-----------+-------+----------+

#### ORC

In [0]:
orc_path='/tmp/ch-5/orc'
df.write.format("orc").mode('overwrite').save(orc_path)

orc_to_delta_path = '/tmp/ch-5/orc_to_delta'
spark.read.format("orc").load(orc_path).write.format("delta").mode('overwrite').save(orc_to_delta_path)
orc_to_delta_df=spark.read.format("delta").load(orc_to_delta_path).show()

+----+-----+---+-----------+-------+----------+
year|month|day| customer|sale_id|total_cost|
+----+-----+---+-----------+-------+----------+
2022| 1| 2|Jack Morris| 12PSD| 100.0|
2022| 3| 2| Jim Smith| M12SD| 30.37|
2022| 6| 2| Jill King| K127D| 50.5|
+----+-----+---+-----------+-------+----------+

### Semi-Structured Data

In [0]:
semi_strutured_data = """
[
 {"Name": "Jack Morris","Preference": "daisy"},
 {"Name": "Jill King",  "Preference": "roses"},
 {"Name": "Jim Smith",  "Preference": "sunflowers"}
]
"""

preferences = spark.read.json(sc.parallelize([semi_strutured_data]))
preferences.show()

delta_flower_preference_path = '/tmp/ch-5/delta_preference'
preferences.write.format("delta").mode('overwrite').save(delta_flower_preference_path)
spark.sql("CREATE TABLE Flower_Preference USING DELTA LOCATION '" + delta_flower_preference_path + "'")

+-----------+----------+
 Name|Preference|
+-----------+----------+
Jack Morris| daisy|
 Jill King| roses|
 Jim Smith|sunflowers|
+-----------+----------+

Out[49]: DataFrame[]

### Unstructured data

In [0]:
daisy_df = spark.read.format("binaryFile").load("/databricks-datasets/flower_photos/daisy").limit(1).withColumn("type", lit('daisy'))
roses_df = spark.read.format("binaryFile").load("/databricks-datasets/flower_photos/roses").limit(1).withColumn("type", lit('roses'))
sunflowers_df = spark.read.format("binaryFile").load("/databricks-datasets/flower_photos/sunflowers").limit(1).withColumn("type", lit('sunflowers'))

catalog_df = daisy_df.union(roses_df).union(sunflowers_df).select('type', 'content')
display(catalog_df)    

delta_flower_catalog_path = '/tmp/ch-5/delta_catalog'
catalog_df.write.format("delta").mode('overwrite').save(delta_flower_catalog_path)
spark.sql("CREATE TABLE Flower_Catalog USING DELTA LOCATION '" + delta_flower_catalog_path + "'")

type,content
daisy,"List(/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"
roses,"List(/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"
sunflowers,"List(/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"


Out[50]: DataFrame[]

### Join disparate data types

In [0]:
%sql
-- select * from Flower_Transaction;
select * from Flower_Preference;
-- select * from Flower_Catalog;

Name,Preference
Jack Morris,daisy
Jill King,roses
Jim Smith,sunflowers


In [0]:
tx_df = spark.table("Flower_Transaction")
preference_df = spark.table("Flower_Preference")
catalog_df = spark.table("Flower_Catalog")

customer = tx_df.join(preference_df, tx_df.customer==preference_df.Name).select('Name', 'Preference')
customer_profile = customer.join(catalog_df, customer.Preference==catalog_df.type).drop('type')
display(customer_profile)

Name,Preference,content
Jack Morris,daisy,"List(/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"
Jill King,roses,"List(/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"
Jim Smith,sunflowers,"List(/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated), ...)"


## Data Curation 
* Hard part of refining data to make it analytic ready
* Applying business transformations 
* Data harmonization

#### Data Cleansing
* There can be different error scenarios around 
  * bad parsing, missing records, need for data normalizations for records from different data sources etc

In [0]:
data = """{"a": 1, "b":2, "c":3}|{"a": 11, "b":12, "c":13}|{"a": 1, "b, "c":10}""".split('|') 

corruptDF = (spark.read.option("mode", "DROPMALFORMED").json(sc.parallelize(data))).show() 
corruptDF = (spark.read.option("mode", "PERMISSIVE") 
                                      .option("Corrupt", "_corrupt_record").json(sc.parallelize(data))).show() 

myBadRecords='/tmp/ch_5/bad_records'
corruptDF = (spark.read.option("badRecordsPath",  myBadRecords).json(sc.parallelize(data))).show()

+---+---+---+
 a| b| c|
+---+---+---+
 1| 2| 3|
 11| 12| 13|
+---+---+---+

+--------------------+----+----+----+
 _corrupt_record| a| b| c|
+--------------------+----+----+----+
 null| 1| 2| 3|
 null| 11| 12| 13|
{"a": 1, "b, "c":10}|null|null|null|
+--------------------+----+----+----+

+---+---+---+
 a| b| c|
+---+---+---+
 1| 2| 3|
 11| 12| 13|
+---+---+---+

In [0]:
%fs ls '/tmp/ch_5/bad_records'

path,name,size,modificationTime
dbfs:/tmp/ch_5/bad_records/20220702T211836/,20220702T211836/,0,1656797003676


### UDFs for distributed custom transformations
* Use Native APIs whereever available as they have been battle tested and optimized
* When necessary create your own logic
* Example: pseudoanonymization of personal details

In [0]:
from pyspark.sql.types import StringType
import random

def mask(s):
  return (''.join(random.sample(s,len(s))))

spark.udf.register("maskUDF", mask)

Out[65]: <function __main__.mask(s)>

In [0]:
%sql
Select customer, maskUDF(customer) from Flower_Transaction

customer,maskUDF(customer)
Jack Morris,r okaMrcJis
Jill King,liJling K
Jim Smith,mt hJSiim


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType
masked_udf = udf(mask, StringType())
tx_df = spark.table("Flower_Transaction")

tx_df.select('customer', masked_udf('customer').alias('masked')).show()

+-----------+-----------+
 customer| masked|
+-----------+-----------+
Jack Morris|iMroJs rcka|
 Jill King| gnKilliJ|
 Jim Smith| mtSJi ihm|
+-----------+-----------+

# Data Governance

### Adding Constraints
* Generated colums
* Adding Quality Checks on idividuaal fields

In [0]:
%sql
CREATE TABLE citizens ( 
  id INT, 
  gender STRING, 
  birthDate TIMESTAMP, 
  dateOfBirth DATE GENERATED ALWAYS AS (CAST(birthDate AS DATE)), 
  salary INT 
)
USING DELTA 
PARTITIONED BY (gender) 

ALTER TABLE default.people10m  
ADD CONSTRAINT dateWithinRange CHECK (birthDate > '1900-01-01') 

### Adding Data Priveleges

In [0]:
%sql
-- GRANT SELECT on TABLE ch_5.Flower_Catalog to `user@company.org`; 
-- DENY ALL on TABLE ch_5.Flower_Transaction to `user@company.org`; 

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.SparkException: Table Access Control is not enabled on this cluster.
	at com.databricks.sql.acl.ResolvePermissionManagement.apply(ResolvePermissionManagement.scala:117)
	at com.databricks.sql.acl.ResolvePermissionManagement.apply(ResolvePermissionManagement.scala:34)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$3(RuleExecutor.scala:216)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$2(RuleExecutor.scala:216)
	at scala.collection.LinearSeqOptimized.foldLeft(LinearSeqOptimized.scala:126)
	at scala.collection.LinearSeqOptimized.foldLeft$(LinearSeqOptimized.scala:122)
	at scala.collection.immutable.List.foldLeft(List.scala:91)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1(RuleExecutor.scala:213)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$execute$1$adapted(RuleExecutor.scala:205)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.execute(RuleExecutor.scala:205)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.org$apache$spark$sql$catalyst$analysis$Analyzer$$executeSameContext(Analyzer.scala:301)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$execute$1(Analyzer.scala:294)
	at org.apache.spark.sql.catalyst.analysis.AnalysisContext$.withNewAnalysisContext(Analyzer.scala:196)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:294)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.execute(Analyzer.scala:222)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.$anonfun$executeAndTrack$1(RuleExecutor.scala:184)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:126)
	at org.apache.spark.sql.catalyst.rules.RuleExecutor.executeAndTrack(RuleExecutor.scala:184)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:274)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:331)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:273)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:128)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:80)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:151)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:265)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:265)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:129)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:126)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:118)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:803)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:798)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:695)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.immutable.List.map(List.scala:293)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)


# Data Serving of SSOT
* Spark is of course a choice for reading from Delta Lake
* There are several others listed in the connector page, we will look at a few
* Delta Standalone Reader for JVM: https://docs.delta.io/latest/delta-standalone.html
  * Single-node Java library that can be used to read from and write to Delta tables. 
  * Provides APIs to interact with a table’s metadata in the transaction log
* Delta Rust API for Python: https://github.com/delta-io/delta-rs